In [ ]:
def add_nan_column(df: pd.DataFrame, cols):
    for col in cols:
        df[col + '_is_nan'] = df[col].isna()
    return df


def add_zero_column(df: pd.Dataframe, cols):
    for col in cols:
        df[col + '_is_zero'] = df[col] == 0
    return df


def calc_price_per_night(df: pd.DataFrame):


# Function to impute the prop_location_score2 values that are NaN based on the prop_location_score2 values of the same prop_id
def impute_prop_loc_score2_using_future_score2(df : pd.DataFrame) -> pd.DataFrame:

    # Retrieve the prop_ids that have a change in prop_location_score2 from NaN to a float over time
    has_change_over_time = df.groupby('prop_id')['prop_location_score2'].apply(lambda x: x.diff().notna().any())
    prop_ids_change = has_change_over_time[has_change_over_time == True].index

    # For each prop_id that has a change in prop_location_score2 from NaN to a float over time, impute the NaN values with the first non-NaN value
    for prop_id in prop_ids_change:
        mask = (df['prop_id'] == prop_id) & (df['prop_location_score2'].isnull())
        non_nan_value = df.loc[(df['prop_id'] == prop_id) & (df['prop_location_score2'].notna()), 'prop_location_score2'].values[0]
        df.loc[mask, 'prop_location_score2'] = non_nan_value

    return df


# Impute remaning NaN values using the mean prop_locatioin_score2 values for each prop_location_score1 value
def impute_prop_location_score2_using_mean_score1(df : pd.DataFrame) -> pd.DataFrame:

    # Create an extra column with rounded 'prop_location_score1'
    df['rounded_score1'] = df['prop_location_score1'].round()

    # Calculate the mean of 'prop_location_score2' for each rounded integer value
    mean_score2_by_rounded = df.groupby('rounded_score1')['prop_location_score2'].mean()

    # Impute missing values of 'prop_location_score2' based on rounded integer values
    df['prop_location_score2'].fillna(df['rounded_score1'].map(mean_score2_by_rounded), inplace=True)

    # Drop the extra column 'rounded_score1' if no longer needed
    df.drop('rounded_score1', axis=1, inplace=True)

    return df


def impute_prop_review_score(df : pd.DataFrame) -> pd.DataFrame:

    # Calculate the mean of 'prop_review_score' for each rounded integer value
    mean_review_score_for_starrating = df.groupby('prop_starrating')['prop_review_score'].mean()

    # Impute missing values of 'prop_review_score' based on rounded integer values
    df['prop_review_score'].fillna(df['prop_starrating'].map(mean_review_score_for_starrating), inplace=True)

    return df


def impute_prop_starrating(df: pd.DataFrame):

def impute_query_affinity_score(df: pd.DataFrame):


# Calculate the mean distance per hotel and impute the NaN values with the mean distance
def impute_orig_destination_distance(df : pd.DataFrame) -> pd.DataFrame:
    
    # Calculate the mean distance per hotel
    mean_distance_per_hotel = df.groupby('prop_id')['orig_destination_distance'].mean()
    
    # Impute the NaN values with the mean distance
    df['orig_destination_distance'].fillna(df['prop_id'].map(mean_distance_per_hotel), inplace=True)

    # BUT there are also hotels that contain only NaN values for orig_destination_distance, 
    # for these we impute it with the mean distance for the country_id
    # Use visitor_location_country_id or country_id?

    # Calculate the mean distance per country
    mean_distance_per_country = df.groupby('visitor_location_country_id')['orig_destination_distance'].mean()

    # Impute the NaN values with the mean distance
    df['orig_destination_distance'].fillna(df['visitor_location_country_id'].map(mean_distance_per_country), inplace=True)
    
    return df

def define_target(df: pd.DataFrame):

def one_hot_encode(df: pd.DataFrame, cols):
    for col in cols:
        df = pd.get_dummies(df, columns=[col], prefix=col)
    return df

def delete_id_columns(df: pd.DataFrame):
    for col in df.columns:
        if col.endswith('_id'):
            df = df.drop(col, axis=1)
    return df

In [ ]:
df = add_nan_column(df, ['prop_review_score'])

df = add_zero_column(df, ['prop_review_score', 'prop_starrating'])